In [14]:
from cleaning_tools import *
from scraping_tools import *
import pandas as pd
from joblib import load, dump

In [ ]:
data = pd.read_csv('clean.csv')
data['boolNB_predictions'] = bNB.predict(v_bool.transform(data['clean']))
data['lgr_predictions'] = lgr.predict(v_tfidf.transform(data['clean']))
fn_lgr = data[(data['injury_report'] == 1) & (data['lgr_predictions'] == 0)]
fp_lgr = data[(data['injury_report'] == 0) & (data['lgr_predictions'] == 1)]

In [ ]:
print(len(fn_lgr['tweet'].drop_duplicates()))

for lines in set(fn_lgr['tweet']) - set(fn_bNB['tweet']):
    print(lines + '\n')

In [ ]:
print(len(fp_lgr['tweet'].drop_duplicates()))

for lines in set(fp_lgr['tweet']) - set(fp_bNB['tweet']):
    print(lines + '\n')

In [2]:
data = pd.read_csv('filtered.csv')
samples = pd.read_csv('sampled.csv')

for i in range(len(samples)):
    data.at[samples['Unnamed: 0'][i], 'injury_report'] = samples['injury_report'][i]
    
data.to_csv('filtered.csv', index=False)

In [3]:
data = data[data['injury_report'] == 'x']
data = data[['injury_report', 'tweet']]
data.drop_duplicates(inplace = True)
data.dropna(inplace = True)
data = data.sample(100000)
data['clean'] = data['tweet'].apply(lambda txt: clean_text(txt))

In [4]:
lgr = load('logistic_regression.joblib')
rfbool = load('random_forest_bool.joblib')
rftfidf = load('random_forest_tfidf.joblib')
bNB = load('bernoulliNB.joblib')
kNN = load('kNN.joblib')
svm = load('svm.joblib')
v_tfidf = load('tfidf.joblib')
v_bool = load('bool.joblib')
v_count = load('count.joblib')

In [5]:
data['boolNB_predictions'] = bNB.predict(v_bool.transform(data['clean']))
data['lgr_predictions'] = lgr.predict(v_tfidf.transform(data['clean']))
positives = data[data['lgr_predictions'] == 1]
positives = positives[['injury_report', 'tweet']]
positives.to_csv('positive_samples.csv')

In [6]:
data = pd.read_csv('filtered.csv')
samples = pd.read_csv('positive_samples.csv')

for i in range(len(samples)):
    data.at[samples['Unnamed: 0'][i], 'injury_report'] = samples['injury_report'][i]
    
data.to_csv('filtered.csv', index=False)

In [7]:
# Make sure the duplicate data (example: retweets) picked up is already labeled.

data = pd.read_csv('filtered.csv')

tweets = data[data['injury_report'] != 'x']['tweet'].value_counts(ascending=False)
tweets = tweets[tweets > 1].index

for i in range(len(tweets)):
    
    if '0' in list(data[data['tweet'] == tweets[i]]['injury_report']):
        data.loc[data[data['tweet'] == tweets[i]].index, 'injury_report'] = '0'
        
    elif '1' in list(data[data['tweet'] == tweets[i]]['injury_report']):
        data.loc[data[data['tweet'] == tweets[i]].index, 'injury_report'] = '1'

In [9]:
data = data[['injury_report', 'tweet']]
data.drop_duplicates(inplace = True)
data['injury_report'].value_counts()

x    930975
0     23111
1      2513
Name: injury_report, dtype: int64

In [12]:
filtered_to_clean()